Para esta iteracion, probaremos con distintas formas de segmentacion sobre nuestro dataset para mejorar la capacidad de prediccion de nuestro modelo.

Buildeamos codigo C++
Mover el build generado en la carpeta experimentacion a la carpeta requerida(en este caso experimentacion_precio)

In [ ]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

Importamos dependencias

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame
import ModeloPrecio
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score, mean_absolute_error
from pathlib import Path

Importamos el dataset a utilizar

In [ ]:
df = pd.read_csv('../../data/train.csv')

Definimos un diccionario con dos modelos distintos. Ambos funcionan de la misma manera pero estan codeados de maneras muy distintas. Los resultados mostrados en el informe son utilizando el v2.

In [ ]:
modelos = {
    'v2': ModeloPrecio.SegmentadoV2,
    'm2_segmentado': ModeloPrecio.ModeloPrecioMetrosCuadradosSeg,
}

Definimos las columnas seleccionadas, las cuales son la mejor combinacion a la hora de predecir SIN segmentacion.
Definimos las distintas segmentaciones a utilizar en esta prueba.

In [ ]:
columnas_seleccionadas = ["escuelascercanas","piscina","usosmultiples","banos","habitaciones","metroscubiertos"]

segmentaciones = [['tipodepropiedad'],['tipodepropiedad', 'provincia'], ['piscina', 'usosmultiples'], ['tipodepropiedad','piscina', 'usosmultiples']]

In [ ]:
KFOLD_K = 5
kf = KFold(n_splits=KFOLD_K, shuffle=True)


result = {}
for segmentacion in segmentaciones:
    rms = []
    rmsle = []
    mae = []
    r2 = []

    for train_index, test_index in kf.split(df):
        df_train, df_test = df.loc[train_index], df.loc[test_index]
        
        mp = modelos['v2'](df_train, columnas_seleccionadas)
        df_predicted = mp.run(df_test, segmentacion)
    
        
        df_predicted['precio'] = df_predicted['precio'].copy().apply(lambda x: x if x > 0 else 0) 
        rms.append(mean_squared_error(df_test["precio"], df_predicted["precio"], squared=False))
        rmsle.append(mean_squared_log_error(df_test["precio"],  df_predicted['precio']))
        r2.append(r2_score(df_test["precio"], df_predicted["precio"]))
        mae.append(mean_absolute_error(df_test["precio"], df_predicted["precio"]))

    rms_result = np.sum(rms) / KFOLD_K 
    rmsle_result = np.sum(rmsle) / KFOLD_K 
    r2_result = np.sum(r2) / KFOLD_K 
    mae_result = np.sum(mae) / KFOLD_K

    result[', '.join(segmentacion)] = [rms_result, rmsle_result, r2_result, mae_result]


out = pd.DataFrame.from_dict(result, orient='index', columns=['RMSE', 'RMSLE', 'R^2', 'MAE']).to_latex(column_format='p{6cm}|r|r|r|r')
print(out)